In [2]:
import os
import json
import pandas as pd
from dotenv import load_dotenv

In [3]:
# from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.1)
# load_dotenv()
# KEY=os.getenv("OPEN_API_KEY")


In [4]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline,AutoModelForCausalLM
# from langchain_community.llms import HuggingFacePipeline
# from langchain_core.prompts import PromptTemplate
# import torch

# # Explicitly load model + tokenizer
# model_id = "google/flan-t5-base"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# # Now create the pipeline using loaded model + tokenizer
# pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=100)

# # Wrap with LangChain
# llm = HuggingFacePipeline(pipeline=pipe)

# # Prompt using new syntax
# prompt = PromptTemplate.from_template("What is the capital of {country}?")
# chain = prompt | llm

# # Run it
# result = chain.invoke({"country": "India"})
# print("Answer:", result)


In [5]:
response_json = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE1="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline,AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch,json
from langchain.chains import LLMChain


c:\Users\Prathyusha\mcqgen\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:


# Explicitly load model + tokenizer
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Now create the pipeline using loaded model + tokenizer
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt1=PromptTemplate(input_variable=["text","number","subject","tone","response_json"],
                                      template=TEMPLATE1)

llm = HuggingFacePipeline(pipeline=pipe)

chain = LLMChain(llm=llm, prompt=prompt1, output_key="quiz")


C:\Users\Prathyusha\AppData\Local\Temp\ipykernel_2616\3940201221.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)
C:\Users\Prathyusha\AppData\Local\Temp\ipykernel_2616\3940201221.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt1, output_key="quiz")


In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
from langchain.chains import SequentialChain

prompt2=PromptTemplate(input_variable=["subject","quiz"],
                                      template=TEMPLATE2)


llm2 = HuggingFacePipeline(pipeline=pipe)
chain2 = LLMChain(llm=llm2, prompt=prompt2, output_key="review")

chain3 = SequentialChain(
    chains=[chain, chain2],
    input_variables=["subject","text", "number", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [11]:
filepath=r"C:/Users/Prathyusha/mcqgen/data.txt"

In [12]:
with open(filepath, "r") as file:
    text = file.read()

In [13]:
output =chain.invoke({
    "subject": "English",
    "text": text,
    "number": 3,
    "tone": "formal",
    "response_json": json.dumps(response_json, indent=4)
})

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


In [14]:
output

{'subject': 'English',
 'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information scien

In [18]:
from datetime import datetime
import logging

LOG_FILE=f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}_app.log"

print(LOG_FILE)


log_path=os.path.join(os.getcwd(), 'logs')
print(log_path)

2025-07-15-12-10-29_app.log
c:\Users\Prathyusha\mcqgen\experiment\logs
